In [9]:
import os
import random

from dotenv import load_dotenv
from openai import OpenAI

In [10]:
load_dotenv(override=True)
client = OpenAI()

## Preparation 

- Define a function that we will use when building our agent.
- It will generate fake weather data:

In [1]:
known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description
We want to use it as a tool for our agent, so we need to describe it

```python
get_weather_tool = {
    "type": "function",
    "name": "<TODO1>",
    "description": "<TODO2>",
    "parameters": {
        "type": "object",
        "properties": {
            "<TODO3>": {
                "type": "string",
                "description": "<TODO4>"
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}
```

In [46]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get weather data for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city whose weather data is required"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

## Define a System and User Prompt 

In [47]:
#system prompt
system_prompt = """You are an intelligent, autonomous assistant capable of reasoning, planning, and taking actions by calling external tools when needed. Your goal is to fulfill user requests accurately and efficiently.

You have access to the following tool:
- `get_weather`: Use this to get current weather information for a given city. It accepts a parameter `query`, which is a string representing the city name.

Your behavior:
- Decide whether a tool call is needed to answer the user’s query.
- If a tool call is necessary, construct the function call with accurate parameters.
- Once you receive the tool's response, interpret it correctly and present the final answer to the user.
- Respond in a clear and informative way, tailored to the user's original intent.
- Ask clarifying questions only when necessary and aim to complete the task in minimal steps.

Constraints:
- Do not fabricate information if a tool call is required to retrieve it.
- Do not ask the user for information that can be directly obtained using available tools.
- Always provide your final answer after using a tool, even if the response is short.

Examples of appropriate tool use:
- User: "What's the weather in London?" → Call `weather` with `query="London"`.
- User: "Should I carry an umbrella in New York?" → Call `weather` with `query="New York"` and interpret rain condition.
- User: "How hot is it in Paris today?" → Call `weather` with `query="Paris"` and extract temperature.

You are proactive, helpful, and reliable. Make decisions like an intelligent assistant that can take initiative.""".strip()


In [48]:
# User prompt
user_question = "What is the weather in New Jersey today?"

## Build chat message

In [68]:
#Agent chat message
chat_messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_question}
               ]

## Create tool list and add weather tool to the list

In [69]:
tools = [get_weather_tool]

## Call the LLM API

In [70]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"city":"New Jersey"}', call_id='call_21L3kjAx0Gqu5Gfz1HbCxrrY', name='get_weather', type='function_call', id='fc_6876b31e64208198b9ed23458a0519ee0cf925ef713baf44', status='completed')]

# Take action based on LLM response

In [71]:
response

Response(id='resp_6876b31d8a5481989e89ecf5afe175920cf925ef713baf44', created_at=1752609565.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFunctionToolCall(arguments='{"city":"New Jersey"}', call_id='call_21L3kjAx0Gqu5Gfz1HbCxrrY', name='get_weather', type='function_call', id='fc_6876b31e64208198b9ed23458a0519ee0cf925ef713baf44', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city whose weather data is required'}}, 'required': ['city'], 'additionalProperties': False}, strict=True, type='function', description='Get weather data for a given city.')], top_p=1.0, background=False, max_output_tokens=None, previous_response_id=None, prompt=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), service_tier='defaul

In [72]:
import json

llm_responses =  response.output
function_names = []
arguments = []

for response_ in llm_responses:
    response_type = response_.type
    response_id = response_.call_id

    if response_type == "function_call":
        function_names.append(response_.name)
        arguments.append(json.loads(response_.arguments))
    

## Check to see if LLM wants to use a tool:
- Call the functions with the returned arguments
- Update the chat messages manually

In [73]:
functions_result = []
for function_name, argument in zip( function_names, arguments):
    function = globals()[function_name]
    result = function(**argument)
    functions_result.append(result)
    print(f"Function call: {function_name}, argument: {argument}\nresult: {result}")

Function call: get_weather, argument: {'city': 'New Jersey'}
result: 11.7


## Call LLM again with updated chat message containing the result from the tool call. 
- Save both the response and the result of the function call:

In [74]:
json.dumps(functions_result[0])

'11.7'

In [75]:
for response_, result in zip(llm_responses, functions_result):
    chat_messages.append(response_)
    
    chat_messages.append({
        "type": "function_call_output",
        "call_id": response_.call_id,
        "output": json.dumps(result),
    })

chat_messages

[{'role': 'system',
  'content': 'You are an intelligent, autonomous assistant capable of reasoning, planning, and taking actions by calling external tools when needed. Your goal is to fulfill user requests accurately and efficiently.\n\nYou have access to the following tool:\n- `get_weather`: Use this to get current weather information for a given city. It accepts a parameter `query`, which is a string representing the city name.\n\nYour behavior:\n- Decide whether a tool call is needed to answer the user’s query.\n- If a tool call is necessary, construct the function call with accurate parameters.\n- Once you receive the tool\'s response, interpret it correctly and present the final answer to the user.\n- Respond in a clear and informative way, tailored to the user\'s original intent.\n- Ask clarifying questions only when necessary and aim to complete the task in minimal steps.\n\nConstraints:\n- Do not fabricate information if a tool call is required to retrieve it.\n- Do not ask th

Now chat_messages contains both the call description (so it keeps track of history) and the results

Let's make another call to the model:



In [76]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)

In [77]:
response.output

[ResponseOutputMessage(id='msg_6876b3330e088198912d396a469f42770cf925ef713baf44', content=[ResponseOutputText(annotations=[], text='The current temperature in New Jersey is 11.7°C. If you need more specific weather details like conditions or a forecast, please let me know!', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]

## Q2. Adding another tool
___
Let's add another tool - a function that can add weather data to our database:

In [43]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [44]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set or update the current temperature for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to update the weather for."
            },
            "temp": {
                "type": "number",
                "description": "The temperature to set for the city (in degrees Celsius)."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}


## Update Your System Prompt


In [78]:
system_prompt = """
You are an intelligent assistant capable of reasoning and calling tools as needed.

You have access to two tools:

1. `get_weather`: Use this tool to get current weather information for a city. It accepts:
   - `query` (string): The city name.

2. `set_weather`: Use this tool to set or update the temperature for a city. It accepts:
   - `city` (string): The name of the city.
   - `temp` (number): The temperature to set, in degrees Celsius.

Use `weather` when the user asks for weather information.
Use `set_weather` when the user wants to define or update the weather for a city.
Respond clearly and helpfully after calling a function.
"""


## Update the tools list

In [79]:
tools.append(set_weather_tool)

## Test calling of the new tool

In [80]:
user_input = "The current temperature in New Jersey is 31 degree celcious"

## Build new chat message

In [81]:
#Agent chat message
chat_messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
               ]

In [82]:
# Call the LLM API
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"city":"New Jersey","temp":31}', call_id='call_SguAy4dtmSJF3mhglWi1comF', name='set_weather', type='function_call', id='fc_6876b59f03f4819bab3646faa42111b00e7ff663d063834b', status='completed')]

In [84]:
llm_responses =  response.output
function_names = []
arguments = []
functions_result = []

for response_ in llm_responses:
    response_type = response_.type
    response_id = response_.call_id

    if response_type == "function_call":
        function_names.append(response_.name)
        arguments.append(json.loads(response_.arguments))


for function_name, argument in zip( function_names, arguments):
    function = globals()[function_name]
    result = function(**argument)
    functions_result.append(result)
    print(f"Function call: {function_name}, argument: {argument}\nresult: {result}")

for response_, result in zip(llm_responses, functions_result):
    chat_messages.append(response_)
    
    chat_messages.append({
        "type": "function_call_output",
        "call_id": response_.call_id,
        "output": json.dumps(result),
    })

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)

response.output

[ResponseOutputMessage(id='msg_6876b6532f54819bb4499b25d6ee38b10e7ff663d063834b', content=[ResponseOutputText(annotations=[], text="I've updated the temperature in New Jersey to 31 degrees Celsius. If you need any more assistance, feel free to ask!", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]

## MCP

In [86]:
!fastmcp --version

2.10.5


## Q4. Simple MCP Server
A simple MCP server from the documentation looks like that:
```python
# weather_server.py
# server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()

```

In [94]:
from fastmcp import FastMCP
import random

# Local state
known_weather_data = {}

# Create FastMCP instance
mcp = FastMCP("Demo 🚀")

# Define tools
@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> str:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

# 🚀 Run the MCP server in Jupyter
await mcp.run_async()



╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                             │
│    📦 Transport:       STDIO                                               │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.10.5                                              │
│    🤝 MCP version:     1.11.0                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

AttributeError: 'OutStream' object has no attribute 'buffer'

In [93]:
mcp.run()

RuntimeError: Already running asyncio in this thread